In [1]:
# sourcery skip: avoid-global-variables
import cProfile
import os
import pstats

import skombo
from skombo.combo_calc import parse_combos_from_csv,COLS,naiive_damage_calc,FD_BOT_CSV_MANAGER

_log = skombo.LOG


test_combo_csv_path = os.path.join(
    skombo.ABS_PATH,
    (skombo.CHARS.AN.lower() + skombo.TEST_COMBOS_SUFFIX),
)


Constants initialized
ABS_PATH: C:\Users\mbradbury\Desktop\pythonZone\skugs\src\skombo
MODULE_NAME: skombo
DATA_PATH: C:\Users\mbradbury\Desktop\pythonZone\skugs\src\skombo\data
CSV_PATH: C:\Users\mbradbury\Desktop\pythonZone\skugs\src\skombo\data\csvs
GAME_DATA_PATH: C:\Users\mbradbury\Desktop\pythonZone\skugs\src\skombo\data\csvs\game_data
CHARACTER_DATA_PATH: C:\Users\mbradbury\Desktop\pythonZone\skugs\src\skombo\data\csvs\game_data\SG2E - Frame Data Bot Data - Characters.csv
FRAME_DATA_PATH: C:\Users\mbradbury\Desktop\pythonZone\skugs\src\skombo\data\csvs\game_data\SG2E - Frame Data Bot Data - Moves.csv
MOVE_NAME_ALIASES_PATH: C:\Users\mbradbury\Desktop\pythonZone\skugs\src\skombo\data\csvs\game_data\SG2E - Frame Data Bot Data - Macros.csv
TEST_DATA_FOLDER: test_data
TEST_COMBOS_SUFFIX: _test_combos.csv
Logger initialized


In [2]:
combo_dfs, combo_damage = parse_combos_from_csv(
    test_combo_csv_path,
)


combo_dfs = [naiive_damage_calc(combo) for combo in combo_dfs]

for i, combo in enumerate(combo_dfs):
    combo_expected_damage = combo_damage[i]
    combo_calculated_damage = combo["summed_damage"].iloc[-1]

    damage_diff = combo_expected_damage - combo_calculated_damage
    damage_diff_percent = round(damage_diff / combo_expected_damage * 100, 3)

    print(f"Combo {i} damage difference: {damage_diff} [{damage_diff_percent}%]")

display(FD_BOT_CSV_MANAGER.dataframes["frame_data"])

========== Parsing combos from csv [C:\Users\mbradbury\Desktop\pythonZone\skugs\src\skombo\annie_test_combos.csv] ==========
Parsing [8] combos from csv
========== Getting frame data bot data ==========
Loading csvs into dataframes
Currect working directory: c:\Users\mbradbury\Desktop\pythonZone\skugs\src\skombo
========== Extracting frame data from fd bot csv ==========
Copied frame data from fd bot csv
Loaded csvs into dataframes
========== Cleaning frame data ==========
Inserted alt name aliases from macros .csv
=== Initial string cleaning ===
Replaced [-] and [ ] with [NaN]
Replaced empty strings with [NaN]
Removed newlines from columns: ['guard', 'properties', 'damage', 'meter_on_hit', 'on_hit_adv', 'on_block_adv', 'startup', 'active', 'recovery', 'hitstun', 'hitstop', 'on_pushblock', 'footer']
Removed [+] and [±] from columns: ['on_hit_adv', 'on_block_adv', 'startup', 'active', 'hitstun', 'on_pushblock']
Removed [%] from [meter] column
Split [alt_names] by [\n]
Split [footer] by 

,character,move_name,alt_names,guard,properties,damage,meter,on_hit,on_block,startup,active,recovery,hitstun,blockstun,hitstop,on_pushblock,footer,thumbnail_url,footer_url
0,ANNIE,5LP,MACRO_5LP,Mid,Chains Into Self,"300, [300 (35)]",2.50%,+1,+4 [+10],6,2,14,16,19,7,+11 to +12,"- Can chain into itself once, or into 2LP.\n- ...",https://wiki.gbl.gg/images/7/73/SG_ann_slp_hb.png,https://wiki.gbl.gg/images/0/0f/SG_ann_slp1_fd...
1,ANNIE,5LP X2,MACRO_5LP_2ND,Mid,-,"300, [300 (35)]",2.50%,+1,+4 [+10],"11 (whiff), 6 (hit/block)",2,14,16,19,7,+11 to +12,NaN,https://wiki.gbl.gg/images/7/73/SG_ann_slp_hb.png,https://wiki.gbl.gg/images/3/35/SG_ann_slp2_fd...
2,ANNIE,5MP,MACRO_5MP,Mid,-,"500, [450 (45)]",7.50%,-2,-11 [-4],12,2,26,25,16,9,-1 to ±0,NaN,https://wiki.gbl.gg/images/3/33/SG_ann_smp_hb.png,https://wiki.gbl.gg/images/3/33/SG_ann_smp_hb.png
3,ANNIE,5HP,MACRO_5HP,Mid,Chains Into Self,"700, [450 x3 (45 x3)]",4.5%,±0,-11 [-9],16,2,33,34,23,11,-8 to -7,- See also 5HP X2.,https://wiki.gbl.gg/images/5/51/SG_ann_shp_hb.png,https://wiki.gbl.gg/images/c/c2/SG_ann_shp_fd.png
4,ANNIE,5HP X2,HP X2\nS.HP X2\nSHP X2\n5.HP X2\n5HPX2\nHPX2\n...,High,-,"600, [650 (70)]",4.5%,Soft KD,-6 [+13],19,2,21,27,16,11,+4 to +5,- Airborne on frame 7.\n- Can use the airborne...,https://wiki.gbl.gg/images/5/51/SG_ann_shp_hb.png,https://wiki.gbl.gg/images/1/1d/SG_ann_shp2_fd...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,VALENTINE,COUNTERVENOM,COUNTER VENOM\nCV\n421KK\n421+KK\nRDP KK\nRDP+...,-,-,2250 (300),-200%,KD,45,0,25,25,30,-,"(0), 7, 51",-,"- See ""COUNTER"" for the non vial loaded versio...",https://wiki.gbl.gg/images/d/de/SG_val_rdpkk_v...,https://wiki.gbl.gg/images/4/40/SG_val_rdpkk_v...
891,VALENTINE,DEAD ON ARRIVAL,MACRO_214PP\nDOA,Mid,"Invuln. (Full),\n55% Min. Scaling,\n Hit Grab","0 x2, 1000, 4000 (250)",-300%,KD,-42,4 + 10,2,64,-,23,"(8), N/A (8 on block)",-,- Hold any button to shorten the animation.\n-...,https://wiki.gbl.gg/images/c/cb/SG_val_qcbpp_h...,https://wiki.gbl.gg/images/0/0a/SG_val_qcbpp_f...
892,VALENTINE,FORBIDDEN PROCEDURE,214MM\n214 MM\n214+MM\nQCBMM\nQCB MM\nQCB+MM\n...,Mid,"Invuln. (Full),\n55% Min. Scaling",4500 (300),-500%,KD,-20,8 + 4,6,49,19,22,"(9), 25 (18 on block)",-,- Revives the corpse of a fallen teammate whil...,https://wiki.gbl.gg/images/8/86/SG_val_qcbtag_...,https://wiki.gbl.gg/images/b/b7/SG_val_qcbtag_...
893,VALENTINE,TAUNT,CHOCOGLYCEMIA\nCHOCOLATE\nCHOCOLATE BAR,-,-,-,0%,-,-,-,-,101,-,-,-,-,- Restores 5 red HP.,https://wiki.gbl.gg/images/4/4c/SG_val_taunt.png,-
